In [17]:
import pandas as pd
import re
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time

In [18]:
# Hàm thu thập danh sách các cầu thủ từ một đội bóng
def extract_players(soup, team):
    
    returner_df = pd.DataFrame(columns = ['team', 'player', 'link'])
    links = []
    players = []
    team_list = []
    
    player_pattern = re.compile(r'(?<=\">).+(?=</a>)')
    link_pattern = re.compile(r'(?<=\ href=").+(?=" id=)')
    

    for each1 in soup.find_all('div', class_ = 'di nowrap'): #, attrs={'class': 'spielprofil_tooltip tooltipstered'}
        
        each2 = each1.find('span', class_ = 'hide-for-small')
        
        if each2 is None:
            continue
        
        each = each2.find('a')
        
        if each is None:
            continue

        try:
            link = each['href']
            player = each.text
            players.append(player)
            links.append(link)
            team_list.append(team)
            '''
            player = re.findall(player_pattern, str(each))[0]
            link = re.findall(link_pattern, str(each))[0]
            player = player.strip()
            link = link.strip()
            players.append(player)
            links.append(link)
            team_list.append(team['teams'])
            '''

        except:
            None
    
    
    returner_df['team'] = team_list
    returner_df['player'] = players
        
    returner_df['link'] = links
    returner_df = returner_df.drop_duplicates(subset='link', keep="last")

    return returner_df

In [19]:
def get_soup(URL):
    
    chrome_options = Options()
    chrome_options.add_argument('--window-size=1,1')
    
    
    driver = webdriver.Chrome(ChromeDriverManager().install(), options = chrome_options)
    wait = WebDriverWait(driver, 10)
    driver.get(URL)
    if len(driver.find_elements_by_css_selector("[name=ROBOTS]")) > 0:
        driver.get(URL)
    soup = BeautifulSoup(driver.page_source)
    driver.close()
    return soup

In [20]:
# Hàm thu thập danh sách cầu thủ từ danh sách các đội bóng
def get_players(teams_df):
    
    players_detail_df = pd.DataFrame()
    
    for each in range(teams_df.shape[0]):
        
        URL = 'https://www.transfermarkt.co.uk' + teams_df.iloc[each]['link']
        print(URL)
        soup = get_soup(URL)
        players = extract_players(soup, teams_df.iloc[each]['teams'])
        print(players)
        
        players_detail_df = pd.concat([players_detail_df, players], axis=0, sort=True)
        print("Complete tm " + teams_df.iloc[each]['teams'])
        
    return players_detail_df

In [21]:
transfermkt_teams = pd.read_csv('C:/Users/Dell/Downloads/Work/TTDL/Đồ án/Data/Obtaining_players_and_teams/teams_transfermarket.csv')

In [22]:
for i in range(transfermkt_teams.shape[0]):
    print(transfermkt_teams.iloc[i]['teams'])

Manchester United
West Ham United
Leicester City
Aston Villa
Wolverhampton Wanderers
Southampton FC
Brighton &amp; Hove Albion
Everton FC
Nottingham Forest
Leeds United
Crystal Palace
Fulham FC
AFC Bournemouth
Brentford FC
Liverpool FC
Newcastle United
Manchester City
Arsenal FC
Tottenham Hotspur
Chelsea FC
Borussia Mönchengladbach
VfL Wolfsburg
TSG 1899 Hoffenheim
1.FC Union Berlin
FC Augsburg
VfB Stuttgart
1.FSV Mainz 05
1.FC Köln
Hertha BSC
FC Schalke 04
VfL Bochum
SV Werder Bremen
Eintracht Frankfurt
SC Freiburg
Borussia Dortmund
Bayer 04 Leverkusen
Bayern Munich
RB Leipzig
Real Sociedad
Villarreal CF
Real Betis Balompié
Athletic Bilbao
Valencia CF
Sevilla FC
Celta de Vigo
CA Osasuna
Girona FC
Rayo Vallecano
RCD Mallorca
UD Almería
Real Valladolid CF
Cádiz CF
Elche CF
RCD Espanyol Barcelona
Getafe CF
Atlético de Madrid
Real Madrid
FC Barcelona
AS Roma
SS Lazio
ACF Fiorentina
US Sassuolo
Torino FC
Udinese Calcio
Bologna FC 1909
AC Monza
FC Empoli
US Lecce
Spezia Calcio
Hellas Verona

In [23]:
players = get_players(transfermkt_teams)

https://www.transfermarkt.co.uk/manchester-united/startseite/verein/985
                 team              player  \
0   Manchester United        David de Gea   
1   Manchester United        Jack Butland   
2   Manchester United          Tom Heaton   
3   Manchester United   Lisandro Martínez   
4   Manchester United      Raphaël Varane   
5   Manchester United       Harry Maguire   
6   Manchester United     Victor Lindelöf   
7   Manchester United          Phil Jones   
8   Manchester United         Teden Mengi   
9   Manchester United           Luke Shaw   
10  Manchester United      Tyrell Malacia   
11  Manchester United    Brandon Williams   
12  Manchester United         Diogo Dalot   
13  Manchester United   Aaron Wan-Bissaka   
14  Manchester United            Casemiro   
15  Manchester United     Scott McTominay   
16  Manchester United   Christian Eriksen   
17  Manchester United     Marcel Sabitzer   
18  Manchester United                Fred   
19  Manchester United   Donn

KeyboardInterrupt: 

In [31]:
print(players)

                                         link       player  \
1          /david-de-gea/profil/spieler/59377    D. de Gea   
3         /jack-butland/profil/spieler/128899   J. Butland   
5            /tom-heaton/profil/spieler/34130    T. Heaton   
7    /lisandro-martinez/profil/spieler/480762  L. Martínez   
9       /raphael-varane/profil/spieler/164770    R. Varane   
..                                        ...          ...   
45       /aidan-simmons/profil/spieler/867763   A. Simmons   
47        /kusini-yengi/profil/spieler/708099     K. Yengi   
49    /nathanael-blair/profil/spieler/1023268     N. Blair   
51    /zachary-sapsford/profil/spieler/703657  Z. Sapsford   
53  /alexander-badolato/profil/spieler/957230  A. Badolato   

                        team  
1          Manchester United  
3          Manchester United  
5          Manchester United  
7          Manchester United  
9          Manchester United  
..                       ...  
45  Western Sydney Wanderers  
47  Wes

In [32]:
players.to_csv('C:/Users/Dell/Downloads/Work/TTDL/Đồ án/Data/Obtaining_players_and_teams/players_transfermarket.csv')